In [8]:
# libraries
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

%store -r Kelmarsh_df Penmanshiel_df

In [1]:
def model_linear_data(data, dependent_var, independent_var, n_lags):

    data[f"{dependent_var}_lag_{n_lags}"] = data[f'{dependent_var}'].shift(n_lags)
    data = data.dropna()

    # Prepare data for modeling
    X = data[[f'{independent_var}', f"{dependent_var}_lag_{n_lags}"]]
    y = data[f"{dependent_var}"]

    # Train-test split (80:20)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

In [3]:
def model_errors (results_df, model, y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    results_df.loc[f'{model}', 'MSE'] = '{:.6f}'.format(mse)
    results_df.loc[f'{model}', 'RMSE'] = '{:.6f}'.format(rmse)
    results_df.loc[f'{model}', 'MAE'] = '{:.6f}'.format(mae)
    results_df.loc[f'{model}', 'MAPE'] = '{:.6f}'.format(mape)

    return results_df

In [4]:
def model_linear(X_train, y_train, X_test):
    # make linear prediction
    linear_reg = LinearRegression()
    linear_reg.fit(X_train, y_train)
    y_pred_linear = linear_reg.predict(X_test)

    return y_pred_linear

In [5]:
def final_model(data, demand, independent_var, n_lags):
    # create results df
    results_df = pd.DataFrame(
        index=['linear regression'],
        columns=['MSE', 'RMSE', 'MAE', 'MAPE']
        )

    # preprocess data
    linear_x_train, linear_x_test, linear_y_train, linear_y_test = model_linear_data(data, demand, independent_var, n_lags)

    # predictions
    linear_pred = model_linear(linear_x_train, linear_y_train, linear_x_test)

    # errors
    model_errors(results_df, 'linear regression', linear_y_test, linear_pred)

    return results_df

In [9]:
final_model(Kelmarsh_df['1'], 'Energy Export (kWh)', 'Long Term Wind (m/s)', 24)

/tmp/ipykernel_1808/3599282347.py:4: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


,MSE,RMSE,MAE,MAPE
linear regression,4602.476194,67.841552,49.552208,inf
